In [1]:
import pandas as pd
import json
import string
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset

/opt/anaconda3/envs/python_main/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
file_path = "/Users/moroz/Documents/projects/пшеница/stack.json"

with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

df = pd.DataFrame(data)

df.head()

,название,Культура,Группа,Код сорта,Описание,Автор(ы),Номер заявки на допуск,Заявители,Дата регистрации заявки на допуск,Год включения в реестр допущенных,...,Сумма пошлины за текущий (9) год поддержания,Сумма пошлины за текущий (8) год поддержания,Сумма пошлины за текущий (28) год поддержания,Сумма пошлины за текущий (14) год поддержания,Сумма пошлины за текущий (22) год поддержания,Сумма пошлины за текущий (23) год поддержания,Сумма пошлины за текущий (19) год поддержания,Сумма пошлины за текущий (26) год поддержания,Сумма пошлины за текущий (20) год поддержания,Сумма пошлины за текущий (27) год поддержания
0,100 ЛЕТ ТАССР,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,8058807,Родословная- (Радуга х О-152/03-7(2) х (Радуга...,АСХАДУЛЛИН ДАМИР ФИДУСОВИЧ; АСХАДУЛЛИН ДАНИЛ Ф...,82178,ФГБУН 'ФЕДЕРАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ ЦЕНТР 'КА...,03 ноября 2020г.,2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,АВИАДА,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,9809918,Родословная- внутривидовая гибридизация озимог...,ЛЕОНОВА ТАТЬЯНА АЛЕКСЕЕВНА; НОВОХАТИН ВЛАДИМИР...,35123,ФГБУН ФЕДЕРАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ ЦЕНТР ТЮМЕ...,09 января 2001г.,2004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,АГАТА,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,8853479,Родословная- Артемовка х Мильтурум 63. Включен...,АНТОШИНА ОЛЬГА АЛЕКСЕЕВНА; БАРКОВСКАЯ ТАТЬЯНА ...,56903,ФГБНУ 'ФЕДЕРАЛЬНЫЙ ИССЛЕДОВАТЕЛЬСКИЙ ЦЕНТР 'НЕ...,10 ноября 2011г.,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,АГРОНОМИЧЕСКАЯ 5,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,7953809,"Родословная- создан с участием сортов Соната, ...",АЙДАРОВ АМАНЖОЛ НУРЖАН УЛЫ; ГЛАДКИХ МАРИНА СЕР...,82007,ФГБОУ ВО 'ОМСКИЙ ГОСУДАРСТВЕННЫЙ АГРАРНЫЙ УНИВ...,23 октября 2020г.,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,АГРОС,Пшеница мягкая яровая (Triticum aestivum L.),Зерновые,7852652,Родословная- Злата x Московская 56. Включен в ...,ВОРОНОВ СЕРГЕЙ ИВАНОВИЧ; ДАВЫДОВА НАТАЛЬЯ ВЛАД...,84044,ООО 'НАДЕЖДА' Г. МОСКВА,05 июля 2021г.,2024,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import re


df_desc_not_str = df['Описание'].copy()
df_desc_not_str = pd.DataFrame(df_desc_not_str)
df_desc_not_str = df_desc_not_str.dropna()
df_desc_not_str = df_desc_not_str.reset_index(drop=True)
    
df_desc_not_str

,Описание
0,Родословная- (Радуга х О-152/03-7(2) х (Радуга...
1,Родословная- внутривидовая гибридизация озимог...
2,Родословная- Артемовка х Мильтурум 63. Включен...
3,"Родословная- создан с участием сортов Соната, ..."
4,Родословная- Злата x Московская 56. Включен в ...
...,...
807,Родословная- 1629/02 х Терра. Включён в Госрее...
808,Родословная- (Терра х Леукурум 459) х Крупинка...
809,Родословная- Харьковская 32 х Линия 841/00. Вк...
810,Родословная- Линия 87/03 х Линия 1816/02. Вклю...


In [ ]:
c = 0
with open('data_desc.txt', "w", encoding="utf-8") as file:
    for i in df_desc_not_str['Описание']:
        if str(i) != 'nan':
            c += 1 
            file.write(str(c) + '. ')
            file.write(str(i))
            file.write('\n')

In [ ]:
import pandas as pd

df_desc_str = pd.DataFrame(columns=['Описание'])

file_path = r"structured_with_pedigree.txt"

with open(file_path, "r", encoding="utf-8") as file:
    lines = file.readlines()

descriptions = []
current_description = []

for line in lines:
    stripped_line = line.strip()
    if stripped_line.startswith("Описание"):
        if current_description:
            descriptions.append({'Описание': " ".join(current_description)})
            current_description = []
    else:
        current_description.append(stripped_line)

if current_description:
    descriptions.append({'Описание': " ".join(current_description)})

df_desc_str = pd.DataFrame(descriptions)

df_desc_str

,Описание
0,Родословная: (Радуга х О-152/03-7(2) х (Радуга...
1,Родословная: внутривидовая гибридизация озимог...
2,Родословная: Артемовка х Мильтурум 63 Госреест...
3,"Родословная: создан с участием сортов Соната, ..."
4,Родословная: Злата x Московская 56 Госреестр: ...
...,...
807,Родословная: инд Госреестр: Включён в Госреест...
808,Родословная: инд Госреестр: Госреестр по Север...
809,Родословная: инд Госреестр: Включён в Госреест...
810,Родословная: инд Госреестр: Включён в Госреест...


In [6]:
df_desc_str = df_desc_str.rename(columns={'Описание': 'structured'})
df_desc_not_str = df_desc_not_str.rename(columns={'Описание': 'unstructured'})

In [7]:
df_concat = pd.concat([df_desc_str, df_desc_not_str], axis=1)


In [9]:
df_concat.head()

,structured,unstructured
0,Родословная: (Радуга х О-152/03-7(2) х (Радуга...,Родословная- (Радуга х О-152/03-7(2) х (Радуга...
1,Родословная: внутривидовая гибридизация озимог...,Родословная- внутривидовая гибридизация озимог...
2,Родословная: Артемовка х Мильтурум 63 Госреест...,Родословная- Артемовка х Мильтурум 63. Включен...
3,"Родословная: создан с участием сортов Соната, ...","Родословная- создан с участием сортов Соната, ..."
4,Родословная: Злата x Московская 56 Госреестр: ...,Родословная- Злата x Московская 56. Включен в ...


In [ ]:
def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_concat['unstructured_clean'] = df_concat['unstructured'].apply(clean_text)
df_concat['structured_clean'] = df_concat['structured'].apply(lambda x: re.sub(r'\s+', ' ', str(x)).strip())

In [14]:
data_dict = {
    "input_text": df_concat["unstructured_clean"].tolist(),
    "target_text": df_concat["structured_clean"].tolist()
}


In [ ]:
dataset = Dataset.from_dict(data_dict)
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]